In [2]:
# Imports

import os
import re
import gzip
import pandas as pd
import numpy as np

# Data Container and Analysis
import anndata as ad
import scanpy as sc
import squidpy as sq

# Machine Learning and Visualization
from IPython.display import display
import matplotlib.pyplot as plt
plt.style.use("default")

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import umap

# CellPLM
from CellPLM.pipeline.cell_type_annotation import CellTypeAnnotationPipeline
from CellPLM.pipeline.cell_embedding import CellEmbeddingPipeline

# Environment Configuration (CUDA check)
import torch
if not torch.cuda.is_available():
    torch_load_old = torch.load
    def torch_load_cpu(*args, **kwargs):
        kwargs['map_location'] = torch.device('cpu')
        return torch_load_old(*args, **kwargs)
    torch.load = torch_load_cpu
    print("Running on CPU mode (CUDA not available).")

/Users/jleto/Library/Python/3.12/lib/python/site-packages/scanpy/_utils/__init__.py:33: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  from anndata import __version__ as anndata_version
/Users/jleto/Library/Python/3.12/lib/python/site-packages/scanpy/__init__.py:24: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  if Version(anndata.__version__) >= Version("0.11.0rc2"):
/Users/jleto/Library/Python/3.12/lib/python/site-packages/scanpy/readwrite.py:16: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  if Version(anndata.__version__) >= Version("0.11.0rc2"):
/Users/jleto/Library/Python/3.12/lib/python/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using t

Running on CPU mode (CUDA not available).


### 1. Gene Expression Data Loading and Merging  

This section loads all spatial transcriptomics files and merges them into one unified gene expression matrix for better downstream analysis.  

The dataset is provided as multiple files, each representing a separate **Visium spatial transcriptomics slide**. Each slide captures **RNA expression patterns** across a thin physical **tissue section**.

These RNA expression patterns show which genes are active and how strongly in different parts of the tissue,  since when a cell uses a gene, it produces mRNA copies of it, *that* is gene expression.

With this technique, RNA molecules released from the tissue bind to barcoded capture spots printed on the slide.  
Each spot records which genes were expressed, basically turning the slide into a grid of thousands of tiny RNA "sensors".  

Because each Visium slide can only capture a limited physical area, the process must be repeated for new sections of tissue, so the dataset comes as multiple files, one for each physical slide (or tissue section).  

Each .txt.gz file contains a **gene expression matrix** for its slide:  
- **Rows** represent genes.  
- **Columns** represent spatial spots (barcoded capture locations on the slide).  
- Each value shows how much RNA from that gene was detected at that spot.  

Each file will be loaded as an AnnData object. The cells below will handle file paths, load counts and coordinates, enforce alignment, and create one single AnnData object for the entire multi-sample dataset.

In [3]:
# Load and Inspect Gene Expression Matrices

import os
import pandas as pd
from IPython.display import display

# Configuration
data_dir = "GSE253975_data"

# Check Directory Contents
print(f"Checking Directory: {data_dir}")

try:
    all_files = [f for f in os.listdir(data_dir) if not f.startswith('.')]
    print(f"Total files found: {len(all_files)}")
    print(all_files)
    
    # Filter only for matrix text files
    txt_files = [f for f in all_files if f.endswith('.txt.gz')]
    print(f"Txt files found: {len(txt_files)}")
    print(txt_files)
    
    if not txt_files:
        print("No .txt.gz files found.")
    else:
        # Pick the first file to test
        test_file = txt_files[0]
        print(f"\n Inspecting First File: {test_file}")
        file_path = os.path.join(data_dir, test_file)

        try:
            df = pd.read_csv(file_path, sep=r'\s+', index_col=0, nrows=5, engine='python') # r'\s+' handles single spaces, tabs, or multiple spaces
            
            print(f"Shape: {df.shape}")
            print(f"Columns (First 5): {list(df.columns)[:5]}")
            
            # Display head for visual check
            print("\nHead of data:")
            display(df.head(2))
            
        except Exception as e:
            print(f"Error reading file: {e}")

except FileNotFoundError:
    print(f"Directory '{data_dir}' not found.")

Checking Directory: GSE253975_data
Total files found: 19
['GSM8031369_T2465.txt.gz', 'GSM8031368_V11U14-042-C1.json.gz', 'GSM8031367_V11U14-040-A1.json.gz', 'GSM8031363_T2791.txt.gz', 'GSM8031364_V11U14-040-C1.json.gz', 'GSM8031361_V11U14-042-A1.json.gz', 'GSM8031361_T0081.txt.gz', 'GSM8031364_T5498.txt.gz', 'GSM8031362_V11U14-042-D1.json.gz', 'GSM8031368_T3799.txt.gz', 'GSM8031366_V11U14-044-C1.json.gz', 'GSM8031365_V11U14-044-A1.json.gz', 'GSM8031362_T3870.txt.gz', 'GSM8031363_V11U14-042-B1.json.gz', 'combined_metadata.json', 'GSM8031367_T4424.txt.gz', 'GSM8031366_T5359.txt.gz', 'GSM8031369_V11U14-040-B1.json.gz', 'GSM8031365_T4839.txt.gz']
Txt files found: 9
['GSM8031369_T2465.txt.gz', 'GSM8031363_T2791.txt.gz', 'GSM8031361_T0081.txt.gz', 'GSM8031364_T5498.txt.gz', 'GSM8031368_T3799.txt.gz', 'GSM8031362_T3870.txt.gz', 'GSM8031367_T4424.txt.gz', 'GSM8031366_T5359.txt.gz', 'GSM8031365_T4839.txt.gz']

 Inspecting First File: GSM8031369_T2465.txt.gz
Shape: (5, 958)
Columns (First 5): ['

,"""T2465_AAACCGTTCGTCCAGG.1""","""T2465_AAACGAGACGGTTGAT.1""","""T2465_AAACTGCTGGCTCCAA.1""","""T2465_AAAGGCTCTCGCGCCG.1""","""T2465_AAAGGGATGTAGCAAG.1""","""T2465_AAATACCTATAAGCAT.1""","""T2465_AAATCGTGTACCACAA.1""","""T2465_AAATGGTCAATGTGCC.1""","""T2465_AAATTAACGGGTAGCT.1""","""T2465_AAATTTGCGGGTGTGG.1""",...,"""T2465_TTGTAAGGACCTAAGT.1""","""T2465_TTGTAAGGCCAGTTGG.1""","""T2465_TTGTAATCCGTACTCG.1""","""T2465_TTGTCGTTCAGTTACC.1""","""T2465_TTGTGTATGCCACCAA.1""","""T2465_TTGTGTTTCCCGAAAG.1""","""T2465_TTGTTAGCAAATTCGA.1""","""T2465_TTGTTCAGTGTGCTAC.1""","""T2465_TTGTTGTGTGTCAAGA.1""","""T2465_TTGTTTCACATCCAGG.1"""
"""MIR1302-2HG""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""FAM138A""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Load Individual Gene Expression Matrices Into a Combined AnnData Master Expression Matrix

# Extract unique sample IDs for tracking
sample_ids = sorted(list(set([f.split('_')[0] for f in txt_files])))
print(f"Found {len(sample_ids)} samples: {sample_ids}")

adatas = []

for sample in sample_ids:
    # Find the text file for this sample
    sample_file = [f for f in txt_files if f.startswith(sample)][0]
    file_path = os.path.join(data_dir, sample_file)
    
    print(f"Loading expression matrix: {sample_file}...")
    
    try:
        # Load Data
        df_counts = pd.read_csv(file_path, sep=r'\s+', index_col=0, engine='python')
        
        # Create AnnData Object
        # Transpose (.T) is required because AnnData expects Rows=Spots, Cols=Genes
        adata = sc.AnnData(df_counts.T)
        
        if adata.n_obs == 0:
            print(f"Skipping {sample}: Matrix is empty.")
            continue
            
        # Add Metadata so we can split them apart later if needed
        adata.obs['sample_id'] = sample
        
        adatas.append(adata)
        
    except Exception as e:
        print(f"Error loading {sample}: {e}")

# Combine into One Unified Matrix
if adatas:
    # Merge all spots and keeps all genes (outer join)
    adata = ad.concat(adatas, label='batch', keys=sample_ids, join='outer')
    
    # Optimize by converting to sparse matrix
    from scipy import sparse
    if not sparse.issparse(adata.X):
        adata.X = sparse.csr_matrix(adata.X)
            
    print(f"\nMaster Expression Matrix Created!")
    print(f"Total Spots: {adata.n_obs}")
    print(f"Total Genes: {adata.n_vars}")
else:
    print("No valid data loaded.")

Found 9 samples: ['GSM8031361', 'GSM8031362', 'GSM8031363', 'GSM8031364', 'GSM8031365', 'GSM8031366', 'GSM8031367', 'GSM8031368', 'GSM8031369']
Loading expression matrix: GSM8031361_T0081.txt.gz...
Loading expression matrix: GSM8031362_T3870.txt.gz...
Loading expression matrix: GSM8031363_T2791.txt.gz...
Loading expression matrix: GSM8031364_T5498.txt.gz...
Loading expression matrix: GSM8031365_T4839.txt.gz...
Loading expression matrix: GSM8031366_T5359.txt.gz...
Loading expression matrix: GSM8031367_T4424.txt.gz...
Loading expression matrix: GSM8031368_T3799.txt.gz...
Loading expression matrix: GSM8031369_T2465.txt.gz...

Master Expression Matrix Created!
Total Spots: 9989
Total Genes: 36601


### 2. Data Normalization  

This step normalizes raw gene expression counts are made mathematically comparable across all spatial spots and slides.

Each slide can be thought of as its own *small experiment*, which means that the total amount of RNA captured per spot, the **sequencing depth**, will naturally vary.
If one spot has twice as many total reads as another, its raw gene counts will be falsely inflated. 

Normalization corrects for. this by scaling every spot's raw gene counts relative to its total library size, scaling them all up to a common factor (like CP10K)
The log-transform then compresses large numerical differences to make the distrubution of gene counts suitable for linear models like PCA.

In [5]:
# Normalize the Merged Gene Expression Matrix

# Restore raw counts
if "counts" in adata.layers:
    # Raw data was already filtered and saved. Restore the raw data 
    print("Restoring filtered raw counts from adata.layers['counts'] for re-normalization...")
    adata.X = adata.layers["counts"].copy()

    if 'log1p' in adata.uns:
        del adata.uns['log1p']
    
    # The filter steps below are skipped on re-run.
else:
    # First Run Only: Apply filtering and save the structural change 
    
    # Remove genes that are all zero across all spots
    print(f"Genes before filtering: {adata.n_vars}")
    sc.pp.filter_genes(adata, min_cells=1) # 'min_cells=1' removes genes with 0 counts
    print(f"Genes after filtering (non-zero): {adata.n_vars}")
    
    # Save this resulting filtered matrix to the layer for all future re-runs
    print("Saving FILTERED raw counts to adata.layers['counts']...")
    adata.layers["counts"] = adata.X.copy()

# Normalize counts (CP10K)
# Runs on every execution because the code restored the RAW data above
sc.pp.normalize_total(adata, target_sum=1e4)

# Log-transform
sc.pp.log1p(adata)

# Validation & Visualization
print(f"\nNormalized Matrix Shape: {adata.shape}")

print("\nTop 10 Highest Expressed Genes "
"\n(Visual check to highlight the most actively expressed genes across all spots, allowing visualization of real variation in the data instead of empty zero regions):")
gene_totals = adata.X.sum(axis=0).A1 
# Get indices of the top 10 genes
top_10_indices = np.argsort(gene_totals)[::-1][:10]
top_10_genes = adata.var_names[top_10_indices]

# Display the top 10 genes for the first 5 spots
display(pd.DataFrame(
    adata.X[:5, top_10_indices].toarray(),
    index=adata.obs_names[:5],
    columns=top_10_genes
))

Genes before filtering: 36601
Genes after filtering (non-zero): 24377
Saving FILTERED raw counts to adata.layers['counts']...

Normalized Matrix Shape: (9989, 24377)

Top 10 Highest Expressed Genes 
(Visual check to highlight the most actively expressed genes across all spots, allowing visualization of real variation in the data instead of empty zero regions):


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


,"""MT-CO1""","""MT-CO2""","""MT-ND2""","""MT-CO3""","""MT-ND1""","""MT-ATP6""","""MT-ND4""","""MT-ND3""","""MT-CYB""","""MBP"""
"""T0081_AAACGAGACGGTTGAT.1""",5.516871,4.748358,5.625654,5.150930,5.437163,5.304253,5.477811,5.094113,4.827734,2.999016
"""T0081_AAACTGCTGGCTCCAA.1""",5.689521,5.378386,5.411026,5.273538,5.069907,5.196987,5.309722,5.069907,5.273538,2.732947
"""T0081_AAAGGGCAGCTTGAAT.1""",6.672632,6.267799,4.887178,5.980750,6.267799,5.576547,6.959998,4.887178,5.576547,5.576547
"""T0081_AAAGTTGACTCCCGTA.1""",4.855061,5.140794,5.831010,5.362766,5.257926,5.766667,5.257926,5.257926,5.544306,4.169672
"""T0081_AAATACCTATAAGCAT.1""",5.748665,5.599647,5.385423,6.035550,5.385423,5.748665,5.424468,5.163425,5.211942,2.270898


The cell bellow aims to correct for the **batch effects** introduced by combining multiple distinct slides.
batch effects are non-biological differences caused by variations in sample preparation, imaging, or techniucal sequencing depth differences.
Although normalization corrected for spot-level sequencing depth variations, **Harmony** is used to correct the remaining *slide-wide* technical bias.
It requies the input to be lower dimensional embeddings, rather than the full gene expression matrix, because processing the full matrix iteratively would be computationally expensive

In [6]:
# SAVE PREPROCESSED ANNDATA (for Notebook 2)

output_path = "adata_preprocessed.h5ad"

adata.write(
    output_path,
    compression="gzip"   # keeps file smaller
)

print(f"\nSaved preprocessed AnnData to: {output_path}")
print("Shape:", adata.shape)
print("Layers:", list(adata.layers.keys()))
print("Done.")


Saved preprocessed AnnData to: adata_preprocessed.h5ad
Shape: (9989, 24377)
Layers: ['counts']
Done.
